In [24]:
import requests
import json
import os
import urllib.parse
from bs4 import BeautifulSoup, Tag

In [25]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'}

In [26]:
def find_last_h2(element):
    current_element = element
    while True:
        current_element = current_element.previous_element
        if current_element is None:
            return None
        if isinstance(current_element, Tag) and current_element.name == 'h2':
            return current_element

In [27]:
def scrape_matches(url):
    
    filename = url.split('/')[-1] + ".json"
    
    print(f"downloading {filename}")
    
    zapasy = []
    itemprops = ['homeTeam','awayTeam','location']
    r = requests.get(url, timeout=20, headers=headers)
    soup = BeautifulSoup(r.text, "html.parser")
    fotbalboxy = soup.find_all(class_='footballbox')
    for f in fotbalboxy:
        zapas = {}
        zapas['competition'] = urllib.parse.unquote(url.split('/')[-1].replace("_"," "))
        zapas['date'] = f.find(class_='fdate').text.split("(")[0].split('[')[0].replace("\xa0"," ").strip()
        if f.find(class_='fdate').find("sup"):
            zapas['date'] = zapas['date'].replace(f.find(class_='fdate').find("sup").text,"")
        zapas['score'] = f.find(class_='fscore').text.replace("–","-")
        if f.find(class_='fscore').find('sup'):
            zapas['score'] = zapas['score'][:-len(f.find(class_='fscore').find('sup').text)]
        for i in itemprops:
            try:
                zapas[i] = f.find(itemprop=i).text.split('[')[0].strip()
                if i != itemprops[-1]:
                    zapas[f"{i}Country"] = f.find(itemprop=i).find(class_='flagicon').find('img')['alt']
                    urls = f.find(itemprop=i).find(itemprop='name').find_all('a')
                    for u in urls:
                        if not u.find_all('img'):
                            zapas[f"{i}Url"] = u['href'].replace('/wiki/','')
            except:
                print(i)
        zapas['phase'] = find_last_h2(f).text.split('[')[0].strip()
        for d in f.find_all('div'):
            if "Attendance" in d.text:
                zapas['attendance'] = d.text.replace('Attendance: ','')
                if d.find('sup'):
                    zapas['attendance'] = zapas['attendance'][:-len(d.find('sup').text)]
        zapasy.append(zapas)

    with open(os.path.join('data_raw',filename), "w+", encoding="utf-8") as soubor:
        soubor.write(json.dumps(zapasy))

    return(zapasy)

In [28]:
download = {
    'UCL': [
        'https://en.wikipedia.org/wiki/2024%E2%80%9325_UEFA_Champions_League_qualifying_phase_and_play-off_round',
        'https://en.wikipedia.org/wiki/2024%E2%80%9325_UEFA_Champions_League',
        
    ],
    'EL': [
        'https://en.wikipedia.org/wiki/2024%E2%80%9325_UEFA_Europa_League_qualifying_phase_and_play-off_round'],
    'UECL': [
        'https://en.wikipedia.org/wiki/2024%E2%80%9325_UEFA_Conference_League_qualifying_phase_and_play-off_round_(Main_Path)',
        'https://en.wikipedia.org/wiki/2024%E2%80%9325_UEFA_Conference_League_qualifying_phase_and_play-off_round_(Champions_Path)'
    ]
}

In [29]:
for competition, pages in download.items():
    for p in pages:
        scrape_matches(p)

downloading 2024%E2%80%9325_UEFA_Champions_League_qualifying_phase_and_play-off_round.json
downloading 2024%E2%80%9325_UEFA_Champions_League.json
downloading 2024%E2%80%9325_UEFA_Europa_League_qualifying_phase_and_play-off_round.json
downloading 2024%E2%80%9325_UEFA_Conference_League_qualifying_phase_and_play-off_round_(Main_Path).json
downloading 2024%E2%80%9325_UEFA_Conference_League_qualifying_phase_and_play-off_round_(Champions_Path).json
